In [ ]:
# Montar Google Drive para guardar modelos y checkpoints
from google.colab import drive
drive.mount('/content/drive')

# Instalar librerías necesarias
!pip install -q kaggle

# Subir tu archivo kaggle.json (credenciales de API)
from google.colab import files
files.upload() # Sube el archivo kaggle.json

# Mover kaggle.json a la ubicación correcta
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Descargar el dataset Food-101
!kaggle datasets download -d kmader/food41
!unzip -q food41.zip -d food101

Mounted at /content/drive


Saving kaggle.json to kaggle.json
Dataset URL: https://www.kaggle.com/datasets/kmader/food41
License(s): copyright-authors
100% 5.29G/5.30G [01:17<00:00, 86.3MB/s]
100% 5.30G/5.30G [01:17<00:00, 73.6MB/s]


In [ ]:
import os
import shutil
from tensorflow.keras.preprocessing.image import ImageDataGenerator

base_dir = '/content/food101/images'

# Verificamos que haya subcarpetas por clase
print("Clases:", len(os.listdir(base_dir)))

Clases: 101


In [ ]:
# Rutas base
images_dir = '/content/food101/images'
meta_dir = '/content/food101/meta/meta'
output_base = '/content/food101_split'

# Crear carpetas /train y /test con subcarpetas por clase
for split in ['train', 'test']:
    for class_name in os.listdir(images_dir):
        os.makedirs(os.path.join(output_base, split, class_name), exist_ok=True)

# Leer archivos de división
with open(os.path.join(meta_dir, 'train.txt'), 'r') as f:
    train_list = [line.strip() for line in f]

with open(os.path.join(meta_dir, 'test.txt'), 'r') as f:
    test_list = [line.strip() for line in f]

# Copiar imágenes a sus carpetas correspondientes
for item in train_list:
    src = os.path.join(images_dir, item + '.jpg')
    dst = os.path.join(output_base, 'train', item.split('/')[0])
    shutil.copy(src, dst)

for item in test_list:
    src = os.path.join(images_dir, item + '.jpg')
    dst = os.path.join(output_base, 'test', item.split('/')[0])
    shutil.copy(src, dst)

print("✅ Imágenes reorganizadas correctamente en /food101_split/train y /test.")


✅ Imágenes reorganizadas correctamente en /food101_split/train y /test.


In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Gl, Dropout
from tensorflow.keras.optimizers import Adam

# Cargar modelo base sin la capa superior
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Congelar capas base

# Agregar capas personalizadas
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(101, activation='softmax')(x)  # 101 clases en Food101

model = Model(inputs=base_model.input, outputs=predictions)

# Compilar el modelo
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Preprocesamiento específico para ResNet50
datagen_train = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.resnet50.preprocess_input,
    rescale=1./255,  # Normaliza las imágenes
    rotation_range=40,  # Aumenta la variabilidad de la orientación de las imágenes
    width_shift_range=0.2,  # Permite mover las imágenes horizontalmente
    height_shift_range=0.2,  # Permite mover las imágenes verticalmente
    shear_range=0.2,  # Rotación arbitraria
    zoom_range=0.2,  # Zoom aleatorio
    horizontal_flip=True,  # Permite reflejar horizontalmente las imágenes
    fill_mode='nearest'  # Cómo rellenar los píxeles vacíos al realizar transformaciones
)

datagen_val = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.resnet50.preprocess_input
)

train_generator = datagen_train.flow_from_directory(
    '/content/drive/MyDrive/food101_split_backup/train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

val_generator = datagen_val.flow_from_directory(
    '/content/drive/MyDrive/food101_split_backup/test',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

Mounted at /content/drive
Found 75750 images belonging to 101 classes.
Found 25250 images belonging to 101 classes.


In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger

# Crear directorio para checkpoints
checkpoint_dir = '/content/drive/MyDrive/food101_checkpoints'
os.makedirs(checkpoint_dir, exist_ok=True)

checkpoint_path = f'{checkpoint_dir}/resnet50_food101_best.h5'

checkpoint = ModelCheckpoint(
    checkpoint_path,
    monitor='val_accuracy',
    save_best_only=True,
    verbose=1,
    save_weights_only=False
)

early_stop = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=3,
    min_lr=1e-7
)

csv_logger = CSVLogger(f'{checkpoint_dir}/training_log.csv', append=True)

callbacks = [checkpoint, early_stop, reduce_lr, csv_logger]


In [ ]:
history = model.fit(
    train_generator,
    epochs=30,
    initial_epoch=10,
    validation_data=val_generator,
    callbacks=callbacks,
    verbose=1
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/30
2368/2368 ━━━━━━━━━━━━━━━━━━━━ 0s 491ms/step - accuracy: 0.2036 - loss: 3.5766

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()



Epoch 1: val_accuracy improved from -inf to 0.55723, saving model to /content/drive/MyDrive/food101_checkpoints/resnet50_food101_best.h5


2368/2368 ━━━━━━━━━━━━━━━━━━━━ 1294s 539ms/step - accuracy: 0.2036 - loss: 3.5764 - val_accuracy: 0.5572 - val_loss: 1.7013 - learning_rate: 1.0000e-04
Epoch 2/30
2368/2368 ━━━━━━━━━━━━━━━━━━━━ 0s 472ms/step - accuracy: 0.4326 - loss: 2.3031
Epoch 2: val_accuracy improved from 0.55723 to 0.59921, saving model to /content/drive/MyDrive/food101_checkpoints/resnet50_food101_best.h5


...

2368/2368 ━━━━━━━━━━━━━━━━━━━━ 1198s 506ms/step - accuracy: 0.5311 - loss: 1.8132 - val_accuracy: 0.6490 - val_loss: 1.3046 - learning_rate: 1.0000e-04
Epoch 10/30
2368/2368 ━━━━━━━━━━━━━━━━━━━━ 0s 462ms/step - accuracy: 0.5418 - loss: 1.7646
Epoch 10: val_accuracy improved from 0.64899 to 0.65184, saving model to /content/drive/MyDrive/food101_checkpoints/resnet50_food101_best.h5


2368/2368 ━━━━━━━━━━━━━━━━━━━━ 1240s 523ms/step - accuracy: 0.5418 - loss: 1.7646 - val_accuracy: 0.6518 - val_loss: 1.2945 - learning_rate: 1.0000e-04
Epoch 11/30
 435/2368 ━━━━━━━━━━━━━━━━━━━━ 14:51 461ms/step - accuracy: 0.5398 - loss: 1.7640

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipython-input-8-2590945388.py", line 1, in <cell line: 0>
    history = model.fit(
              ^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/keras/src/utils/traceback_utils.py", line 117, in error_handler
    return fn(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/keras/src/backend/tensorflow/trainer.py", line 371, in fit
    logs = self.train_function(iterator)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/keras/src/backend/tensorflow/trainer.py", line 219, in function
    opt_outputs = multi_step_on_iterator(iterator)
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/tensorflow/python/util/traceback_utils.py", line 150, in er

TypeError: object of type 'NoneType' has no len()

In [ ]:
def load_model_from_checkpoint():
    from tensorflow.keras.models import load_model

    checkpoint_path = '/content/drive/MyDrive/food101_checkpoints/resnet50_food101_best.h5'

    if os.path.exists(checkpoint_path):
        print("Cargando modelo desde checkpoint...")
        model = load_model(checkpoint_path)
        print("Modelo cargado exitosamente!")
        return model
    else:
        print("No se encontró checkpoint, creando modelo nuevo...")
        return None

In [ ]:
from sklearn.metrics import matthews_corrcoef, classification_report, confusion_matrix
import numpy as np

def calculate_multiclass_mcc(model, test_generator):
    """
    Calcula el Coeficiente de Correlación de Matthews en modo multiclase.
    """
    # Predicciones y verdaderas etiquetas
    test_generator.reset()
    preds = model.predict(test_generator, verbose=1)
    y_pred = np.argmax(preds, axis=1)
    y_true = test_generator.classes

    # MCC multiclase
    mcc = matthews_corrcoef(y_true, y_pred)
    print(f"Matthews Correlation Coefficient (multiclase): {mcc:.4f}")

    # Reporte de clasificación
    labels = list(test_generator.class_indices.keys())
    print("\nReporte de clasificación:")
    print(classification_report(y_true, y_pred, target_names=labels))

    # Matriz de confusión (multiclase)
    cm = confusion_matrix(y_true, y_pred)
    print("\nMatriz de confusión (forma resumida multiclase):")
    print(cm)

    return mcc, cm


In [ ]:
mcc, cm = calculate_multiclass_mcc(model, val_generator)

790/790 ━━━━━━━━━━━━━━━━━━━━ 98s 124ms/step
Matthews Correlation Coefficient: -0.0004

Reporte de clasificación:
                         precision    recall  f1-score   support

              apple_pie       0.00      0.00      0.00       250
         baby_back_ribs       0.01      0.02      0.01       250
                baklava       0.01      0.01      0.01       250
         beef_carpaccio       0.01      0.02      0.02       250
           beef_tartare       0.01      0.01      0.01       250
             beet_salad       0.00      0.00      0.00       250
               beignets       0.01      0.02      0.01       250
               bibimbap       0.00      0.00      0.00       250
          bread_pudding       0.03      0.02      0.02       250
      breakfast_burrito       0.01      0.01      0.01       250
             bruschetta       0.01      0.00      0.01       250
           caesar_salad       0.01      0.01      0.01       250
                cannoli       0.00      0

TypeError: cannot unpack non-iterable numpy.float64 object

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report

# Asegúrate de tener modelo y val_generator disponibles
val_generator.reset()
preds = model.predict(val_generator, verbose=1)
y_pred = np.argmax(preds, axis=1)
y_true = val_generator.classes

# Obtener nombres de las clases ordenadas (según generator)
labels = list(val_generator.class_indices.keys())

# Matriz de confusión multiclase (101x101)
cm = confusion_matrix(y_true, y_pred)

# Visualización
fig, ax = plt.subplots(figsize=(22, 22))  # Ajusta el tamaño según tu pantalla
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)
disp.plot(ax=ax, cmap=plt.cm.Blues, xticks_rotation=90, colorbar=True)

plt.title("Matriz de Confusión - Food101 (101 clases)")
plt.xlabel("Clase Predicha")
plt.ylabel("Clase Verdadera")
plt.grid(False)
plt.tight_layout()
plt.show()

# Reporte de clasificación por clase (opcional)
print("\nReporte de clasificación por clase:")
print(classification_report(y_true, y_pred, target_names=labels))


  2/790 ━━━━━━━━━━━━━━━━━━━━ 1:33:22 7s/step

KeyboardInterrupt: 

In [ ]:
from scipy.stats import mcnemar
from tensorflow.keras.models import load_model

def mcnemar_test_comparison(model1, model2_path, test_generator, model1_name="ResNet50", model2_name="InceptionV3"):
    """
    Compara dos modelos usando la prueba de McNemar

    Args:
        model1: Modelo ya cargado (tu ResNet)
        model2_path: Ruta al modelo .h5 de InceptionV3
        test_generator: Generador de datos de prueba
        model1_name: Nombre del primer modelo
        model2_name: Nombre del segundo modelo
    """

    # Cargar el segundo modelo
    print(f"Cargando {model2_name} desde {model2_path}")
    model2 = load_model(model2_path)

    # Predicciones del primer modelo
    print(f"Obteniendo predicciones de {model1_name}...")
    test_generator.reset()
    predictions1 = model1.predict(test_generator, verbose=1)
    predicted_classes1 = np.argmax(predictions1, axis=1)

    # Predicciones del segundo modelo
    print(f"Obteniendo predicciones de {model2_name}...")
    test_generator.reset()
    predictions2 = model2.predict(test_generator, verbose=1)
    predicted_classes2 = np.argmax(predictions2, axis=1)

    # Obtener etiquetas verdaderas
    true_classes = test_generator.classes

    # Calcular accuracy de cada modelo
    acc1 = np.mean(predicted_classes1 == true_classes)
    acc2 = np.mean(predicted_classes2 == true_classes)

    print(f"\nAccuracy {model1_name}: {acc1:.4f}")
    print(f"Accuracy {model2_name}: {acc2:.4f}")

    # Crear tabla de contingencia para McNemar
    model1_correct = (predicted_classes1 == true_classes)
    model2_correct = (predicted_classes2 == true_classes)

    both_correct = np.sum(model1_correct & model2_correct)
    model1_only = np.sum(model1_correct & ~model2_correct)
    model2_only = np.sum(~model1_correct & model2_correct)
    both_wrong = np.sum(~model1_correct & ~model2_correct)

    print(f"\nTabla de contingencia:")
    print(f"Ambos correctos: {both_correct}")
    print(f"Solo {model1_name} correcto: {model1_only}")
    print(f"Solo {model2_name} correcto: {model2_only}")
    print(f"Ambos incorrectos: {both_wrong}")

    # Test de McNemar
    if model1_only + model2_only > 0:
        # Crear tabla para McNemar (formato específico)
        mcnemar_table = np.array([[both_correct, model2_only],
                                 [model1_only, both_wrong]])

        # Ejecutar test
        result = mcnemar(mcnemar_table, exact=True)
        print(f"\nMcNemar Test:")
        print(f"Estadístico: {result.statistic}")
        print(f"p-value: {result.pvalue:.6f}")

        alpha = 0.05
        if result.pvalue < alpha:
            print(f"Resultado: Los modelos tienen diferencias estadísticamente significativas (p < {alpha})")
            if model1_only > model2_only:
                print(f"       -> {model1_name} es significativamente mejor")
            else:
                print(f"       -> {model2_name} es significativamente mejor")
        else:
            print(f"Resultado: No hay diferencias estadísticamente significativas entre los modelos (p >= {alpha})")
    else:
        print("No hay casos donde los modelos difieran, no se puede aplicar McNemar")

    return {
        'accuracy_model1': acc1,
        'accuracy_model2': acc2,
        'both_correct': both_correct,
        'model1_only': model1_only,
        'model2_only': model2_only,
        'both_wrong': both_wrong,
        'mcnemar_result': result if model1_only + model2_only > 0 else None
    }

# Ejemplo de uso:
# Comparar tu ResNet con tu InceptionV3
# inception_model_path = '/content/drive/MyDrive/tu_modelo_inception.h5'
# results = mcnemar_test_comparison(model, inception_model_path, val_generator,
#                                  model1_name="ResNet50", model2_name="InceptionV3")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

def plot_training_results(history_path):
    """
    Visualiza los resultados del entrenamiento
    """
    with open(history_path, 'rb') as f:
        history = pickle.load(f)

    fig, axes = plt.subplots(2, 2, figsize=(15, 10))

    # Accuracy
    axes[0,0].plot(history['accuracy'], label='Train Accuracy')
    axes[0,0].plot(history['val_accuracy'], label='Val Accuracy')
    axes[0,0].set_title('Model Accuracy')
    axes[0,0].set_xlabel('Epoch')
    axes[0,0].set_ylabel('Accuracy')
    axes[0,0].legend()

    # Loss
    axes[0,1].plot(history['loss'], label='Train Loss')
    axes[0,1].plot(history['val_loss'], label='Val Loss')
    axes[0,1].set_title('Model Loss')
    axes[0,1].set_xlabel('Epoch')
    axes[0,1].set_ylabel('Loss')
    axes[0,1].legend()

    # Learning Rate (si está disponible)
    if 'lr' in history:
        axes[1,0].plot(history['lr'])
        axes[1,0].set_title('Learning Rate')
        axes[1,0].set_xlabel('Epoch')
        axes[1,0].set_ylabel('LR')

    plt.tight_layout()
    plt.show()

# Usar para visualizar resultados
# plot_training_results('/content/drive/MyDrive/food101_checkpoints/training_history.pkl')